In [2]:
import h5py
import os
import numpy as np
import torch
from tqdm import tqdm
from collections import defaultdict
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import json

In [2]:
datanames = ['toys-split','sports-split','beauty-split']
dataname = datanames[0]
data_root = f"/data2/wangzhongren/taolin_project/dataset/{dataname}"
output_root = f"/data2/wangzhongren/taolin_project/data/{dataname}"
sample_filename = f"{dataname}.item"
dataset_types = ['train','valid','test']


In [26]:
# 对item数据集的特征列进行处理，包括填充缺失值，对类别的分割
feat_keys = ['item_id', 'sales_type', 'brand','categories']
sample_path = os.path.join(data_root,sample_filename)
df_feat = pd.read_csv(sample_path, sep='\t', header=0)
df_feat.columns = [col.split(":")[0] for col in df_feat.keys()]
df_feat= df_feat[feat_keys]
# 不对类目特征做额外处理
# for i in range(3):
#     df_feat[f'category_{i+1}'] = df_feat['categories'].str.split(',').str.get(i).str.strip().fillna('Unknown')
# df_feat = df_feat.drop(columns=['categories'])
df_feat['sales_type'] = df_feat['sales_type'].fillna("missing")
df_feat[:3]

,item_id,sales_type,brand,categories
0,0375829695,Home & Kitchen,Dr. Seuss,"'Toys & Games', 'Puzzles', 'Jigsaw Puzzles'"
1,0439855896,Toys & Games,Rock Ridge,"'Toys & Games', 'Novelty & Gag Toys', 'Magic K..."
2,0439893577,missing,Scholastic,"'Toys & Games', 'Pretend Play', 'Dress Up & Pr..."


In [27]:
# 对训练用数据集进行初步处理，label,列名等等，检查是否有空值
df_all = pd.DataFrame()
for dataset_type in dataset_types:
    data_filename = f"{dataname}.{dataset_type}.inter"
    data_path = f"{data_root}/{data_filename}"
    df_data = pd.read_csv(data_path, sep='\t', header=0)
    df_data.columns = [col.split(":")[0] for col in df_data.keys()]
    df_data['label'] = (df_data['rating']>3).astype(int)
    df_data = df_data.drop(columns = ['rating'])
    df_data = df_data.drop(columns = ['timestamp'])
    df_data['dataset_type'] = dataset_type
    df_all = pd.concat([df_all, df_data], ignore_index=True)
df_all[:3]

,user_id,item_id,label,dataset_type
0,AYVR1MQCTNU5D,0375829695,1,train
1,A3CJHKFHHQJP2K,0375829695,0,train
2,A3638FINP26E8N,0375829695,0,train


In [28]:
# 把item特征混合到训练集中
df_all_merged = pd.merge(df_all,df_feat,on='item_id',how='left')
df_all_merged[:3]

,user_id,item_id,label,dataset_type,sales_type,brand,categories
0,AYVR1MQCTNU5D,0375829695,1,train,Home & Kitchen,Dr. Seuss,"'Toys & Games', 'Puzzles', 'Jigsaw Puzzles'"
1,A3CJHKFHHQJP2K,0375829695,0,train,Home & Kitchen,Dr. Seuss,"'Toys & Games', 'Puzzles', 'Jigsaw Puzzles'"
2,A3638FINP26E8N,0375829695,0,train,Home & Kitchen,Dr. Seuss,"'Toys & Games', 'Puzzles', 'Jigsaw Puzzles'"


In [ ]:
item_ids_path = os.path.join(data_root, "item_id.pt")
item_ids_list = torch.load(item_ids_path)
print(f"✅ 加载item_ids_list: {len(item_ids_list)}个items")
print(f"   示例: {item_ids_list[:3]}")
item_to_position = {}
for pos, item_id in enumerate(item_ids_list):
    key = item_id.decode('utf-8') if isinstance(item_id, bytes) else str(item_id)
    item_to_position[key] = pos
df_all_merged['item_position'] = df_all_merged['item_id'].astype(str).map(
    lambda x: item_to_position.get(x, 0)
)

In [29]:
# 使用labelencoder对稀疏特征进行编码
encodered_columns = ['user_id', 'item_id', 'sales_type', 'brand','categories']
lbe = LabelEncoder()
for column in encodered_columns:
    df_all_merged[column] = lbe.fit_transform(df_all_merged[column])
columns = [col for col in df_all_merged.columns if col != 'label'] + ['label']
df_all_merged = df_all_merged[columns]
df_all_merged[:3]

,user_id,item_id,dataset_type,sales_type,brand,categories,label
0,909658,0,train,11,1415,517,1
1,569610,0,train,11,1415,517,0
2,526039,0,train,11,1415,517,0


In [30]:
# 保存为csv和h5
for dataset_type in dataset_types: 
    df_split = df_all_merged[df_all_merged['dataset_type'] == dataset_type].drop(columns=['dataset_type'])
    output_path = f"{output_root}/base_dataset"
    os.makedirs(output_path,exist_ok = True)
    df_split.to_csv(os.path.join(output_path,f'{dataset_type}.csv'),index=False)
    print(f"{dataset_type}.csv saved")
    # with h5py.File(os.path.join(output_path,f'{dataset_type}.h5'), 'w') as f:
    #     f.create_dataset('data', data=df_split.values,dtype='float64')
    # print(f"{dataset_type}.h5 saved")

train.csv saved
valid.csv saved
test.csv saved


In [5]:
item_ids_path = os.path.join(data_root, "item_id.pt")
item_ids_list = torch.load(item_ids_path)
print(f"✅ 加载item_ids_list: {len(item_ids_list)}个items")
print(f"   示例: {item_ids_list[:3]}")

✅ 加载item_ids_list: 56657个items
   示例: ['0375829695', '0439855896', '0439893577']


/tmp/ipykernel_936137/3315848356.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  item_ids_list = torch.load(item_ids_path)


In [3]:
index = torch.load("/data2/wangzhongren/taolin_project/dataset/toys-split/moc_cbsize256_cbdim32_scala7_epoch500_index.pt")
index

/tmp/ipykernel_3091269/3708196371.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  index = torch.load("/data2/wangzhongren/taolin_project/dataset/toys-split/moc_cbsize256

tensor([[ 74, 132, 206,  ..., 223, 252,  89],
        [132,  87,  86,  ..., 207,  86,  87],
        [147,   6,  62,  ...,  82, 167, 213],
        ...,
        [212, 159,  44,  ..., 146, 147, 179],
        [162, 153,  50,  ...,  18, 217, 145],
        [  3,  87,  60,  ..., 187,  86,  87]], device='cuda:7')